In [123]:
import datetime
import time
import TimeEncoder
import pandas as pd

# Generating the DataFrame with dates for test
range_of_dates = pd.date_range(start="2017-01-01 00:00:00", end="2017-01-01 23:59:59", freq='1s')

time_list = []
for i in range(len(range_of_dates)):
    time_list.append(range_of_dates[i].time())

# create a sequence of day numbers
X = pd.DataFrame(index=range_of_dates)
X["day_nr"] = range(len(X))
X["day_of_year"] = X.index.day_of_year
# print("X : ",'\n',type(X['day_nr']))

tester = TimeEncoder.TimeEncoder()
test01= tester.daySin(X['day_nr'])
test02 = tester.dayCos(X['day_nr'])

df1 = pd.DataFrame(test01)
df2 = pd.DataFrame(test02)
df1.rename(columns={"day_nr":"time_sin"}, inplace=True)
df2.rename(columns={"day_nr":"time_cos"}, inplace=True)

df_concat = pd.concat([df1, df2], axis=1)

df_concat['time'] = time_list
df = df_concat.reset_index(drop=True)
# df.to_csv("time_comparison_table.csv")

In [152]:
range_of_year = pd.date_range(start="2021-01-01", end="2021-12-31", freq='D')
print(len(range_of_year))
beta = pd.DataFrame(index=range_of_year)
beta['day_of_year'] = beta.index.day_of_year
beta_sin = tester.yearSin(beta['day_of_year'])
beta_cos = tester.yearCos(beta['day_of_year'])
df_beta_s = pd.DataFrame(beta_sin)
df_beta_s.rename(columns={'day_of_year':'sin_year'}, inplace=True)

df_beta_c = pd.DataFrame(beta_cos)
df_beta_c.rename(columns={'day_of_year':'cos_year'}, inplace=True)
beta_concat = pd.concat([beta,df_beta_s,df_beta_c], axis=1)
beta_concat = beta_concat.reset_index(drop=True)
beta_concat
beta_concat.to_csv('normal_year_table.csv')

365


In [255]:
beta

,day_of_year
2021-01-01,1
2021-01-02,2
2021-01-03,3
2021-01-04,4
2021-01-05,5
...,...
2021-12-27,361
2021-12-28,362
2021-12-29,363
2021-12-30,364


In [297]:
"""인풋 데이터를 받고 함수로 처리"""
"""시간 데이터에서 시간 값만 추출해서 time comparison_table과 비교해
sin cos 값을 리턴하는 함수 만들기"""
import re
from datetime import date

class TimeEncoder:

    # pandas series를 input parameter로 받고 list를 return
    def time2SinCos(time_data : pd.Series) :

        len_data = len(time_data)
        alpha = time_data[0] # 문자열에서 시간이 위치하는 인덱스를 찾기 위한 변수
        time_location = (re.search(r'\d{2}:', alpha).span(0)[0])# 시간 데이터 인덱스

        # 시간별 sin-cos 비교 데이터

        time_comparison_data = pd.read_csv('time_comparison_table.csv')

        sin_list = []
        cos_list = []

        for k in range(len_data):
            temp = time_data[k]
            time_extraction = temp[time_location:time_location+8]
            # print(time_extraction)

            x = time_extraction
            y = int(x[0:2])*3600 + int(x[3:5])*60 + int(x[6:8])
            # print("y : ",y, '\n')
            sin = time_comparison_data.iloc[y].time_sin
            cos = time_comparison_data.iloc[y].time_cos
            sin_list.append(sin)
            cos_list.append(cos)
            # print("sin : ",sin,'\n')
            # print("cos : ",cos,'\n')

        return sin_list, cos_list

    # 365일을 갖는 평년을 위한 메서드
    def day2SinCos(time_data:pd.Series):

        len_day = len(time_data)
        alpha = time_data[0]
        l = (re.search(r'\d{4}-\d{2}-\d{2}', alpha).span(0)[0])# 시간 데이터 인덱스

        normal_year = pd.read_csv('normal_year_table.csv')
        # leap_year = pd.read_csv('leap_year_table.csv') # 윤년

        day_sin =[]
        day_cos = []
        for k in range(len_day):

            temp = time_data[k]

            time_temp1 = date(int(temp[l:l+4]), int(temp[l+5:l+7]),int(temp[l+8:10]))
            time_temp = date(int(temp[l:l+4]),1,1)

            date_delta = time_temp1 - time_temp # 날짜 값 인덱스를 맞춰주기 위해 1더함
            date_delta = date_delta.days + 1 # timedelta type의 변수라 에러 가 있었음

            day_sin.append(normal_year.iloc[date_delta].sin_year)
            day_cos.append(normal_year.iloc[date_delta].cos_year)

        return day_sin, day_cos

In [298]:
venezia_data = pd.read_csv('venezia.csv')
len(venezia_data)
tester03 = venezia_data.datetime[1]
print(tester03[3:6])
re.search(r'\d{4}-\d{2}-\d{2}',venezia_data.datetime[1])
print(venezia_data.datetime[1][0:4])
print(date(1,1,1))


3-0
1983
0001-01-01


In [299]:
venezia_data = pd.read_csv('venezia.csv')
# time2SinCos = TimeEncoder.time2SinCos
# sin_list,cos_list = time2SinCos(venezia_data.datetime)
# len(sin_list)
# print(len(venezia_data)//10000)
# print(len(cos_list))
# print(cos_list)
day2SinCos = TimeEncoder.day2SinCos
sin, cos = day2SinCos(venezia_data.datetime)


# test = day2SinCos
# test
data = pd.read_csv('normal_year_table.csv')
data.iloc[1].sin_year

0.0344216116227457

In [303]:
cos

[0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0344216116227457,
 0.0516196672232537,
 0.0516196672232537,
 0.0516196672232537,
 0.0516196672232537,
 0.0516196672232537]